In [1]:
import os
import glob
import uproot
import mplhep as hep
import numpy as np
import awkward as ak
import boost_histogram as bh
import matplotlib.pyplot as plt
import json

In [2]:
def get_lumi_dict(grl_dir):
    grl_csvs = glob.glob(os.path.join(grl_dir, "*.csv"))
    lumi_dict = {}
    for grl in grl_csvs:
        with open(grl, 'r') as f:
            for i, line in enumerate(f):
                if i == 0: continue

                spline = line.split(",")
                lumi_rec = float(spline[3])
                run_num = int(spline[0])
                lumi_dict[run_num] = lumi_rec

    return lumi_dict

In [3]:
lumi_dict = get_lumi_dict("/cvmfs/faser.cern.ch/repo/sw/runlist/v8/")
runs = np.array(list(lumi_dict.keys()))
runs_2022 = runs[runs < 1e4]
runs_2023 = runs[runs > 1e4]
runs_2023 = runs_2023[runs_2023 < 1.2e4]
runs_2024 = runs[runs > 1.2e4]

In [4]:
low_lumi_runs = []
for run, lumi in lumi_dict.items():
    if lumi < 10: 
        low_lumi_runs.append(run)
print(low_lumi_runs)

[10417, 10419, 10443, 10540, 10572, 10600, 10602, 10747, 10799, 11214, 11461, 11463, 11478, 11480, 11486, 11491, 11706, 7733, 7734, 7802, 7833, 7835, 7836, 7848, 7849, 7971, 7984, 7987, 7988, 7989, 7990, 7998, 8034, 8036, 8037, 8038, 8039, 8040, 8041, 8090, 8096, 8097, 8098, 8099, 8104, 8105, 8106, 8108, 8109, 8110, 8111, 8112, 8113, 8114, 8243, 8247, 8253, 8295, 8651, 8652, 8655, 8658, 8713, 8714, 8734, 8751, 8901, 8904, 8910, 8932, 9048, 9053, 9056, 9066, 9171, 14587, 14588, 14589, 14590, 14597, 14733, 14766, 14776, 15075, 15146, 15181, 15257, 15259, 15260, 15267, 15421, 15455, 15553, 15676, 15678, 15679, 15868, 15975, 15993, 16522, 16648, 16669, 16716]


A list of bad runs can be found on the twiki here: https://twiki.cern.ch/twiki/bin/view/FASER/BadFASERRuns

In [5]:
bad_runs = [{'runs': [15694], "reason": "ATLAS lumi unrealiable"},
            {'runs': [15415, 15418, 15421, 15422, 15424, 15425, 15429], "reason": "TCL6 Collimator settings changed"},
            {'runs': [16100], "reason": "LHC missing B2 trains"},
            {'runs': [16648, 16649], "reason": "Bad filewriter state? (Not sure about this)"},
            {'runs': [16963], "reason": "Missing LHC bunch trains"},
            {'runs': [10417, 10419, 10443, 10540, 10572, 10600, 10602, 10747, 10799, 11214, 11461, 11463, 11478, 11480, 11486, 11491, 11706, 7733, 7734, 7802, 7833, 7835, 7836, 7848, 7849, 7971, 7984, 7987, 7988, 7989, 7990, 7998, 8034, 8036, 8037, 8038, 8039, 8040, 8041, 8090, 8096, 8097, 8098, 8099, 8104, 8105, 8106, 8108, 8109, 8110, 8111, 8112, 8113, 8114, 8243, 8247, 8253, 8295, 8651, 8652, 8655, 8658, 8713, 8714, 8734, 8751, 8901, 8904, 8910, 8932, 9048, 9053, 9056, 9066, 9171, 14587, 14588, 14589, 14590, 14597, 14733, 14766, 14776, 15075, 15146, 15181, 15257, 15259, 15260, 15267, 15421, 15455, 15553, 15676, 15678, 15679, 15868, 15975, 15993, 16522, 16648, 16669, 16716], "reason": "Run has < 10 /pb"}
           ]
    

In [7]:
for i, bad_run in enumerate(bad_runs):
    lost_lumi = sum([lumi_dict[r] for r in bad_run['runs']])
    bad_runs[i]['lumi'] = lost_lumi
print(bad_runs)
with open('bad_runs.json', 'w') as f:
    json.dump(bad_runs, f)

[{'runs': [15694], 'reason': 'ATLAS lumi unrealiable', 'lumi': 19.609}, {'runs': [15415, 15418, 15421, 15422, 15424, 15425, 15429], 'reason': 'TCL6 Collimator settings changed', 'lumi': 2884.161}, {'runs': [16100], 'reason': 'LHC missing B2 trains', 'lumi': 152.603}, {'runs': [16648, 16649], 'reason': 'Bad filewriter state? (Not sure about this)', 'lumi': 248.5}, {'runs': [16963], 'reason': 'Missing LHC bunch trains', 'lumi': 36.218}, {'runs': [10417, 10419, 10443, 10540, 10572, 10600, 10602, 10747, 10799, 11214, 11461, 11463, 11478, 11480, 11486, 11491, 11706, 7733, 7734, 7802, 7833, 7835, 7836, 7848, 7849, 7971, 7984, 7987, 7988, 7989, 7990, 7998, 8034, 8036, 8037, 8038, 8039, 8040, 8041, 8090, 8096, 8097, 8098, 8099, 8104, 8105, 8106, 8108, 8109, 8110, 8111, 8112, 8113, 8114, 8243, 8247, 8253, 8295, 8651, 8652, 8655, 8658, 8713, 8714, 8734, 8751, 8901, 8904, 8910, 8932, 9048, 9053, 9056, 9066, 9171, 14587, 14588, 14589, 14590, 14597, 14733, 14766, 14776, 15075, 15146, 15181, 15257, 

A list of the the different run periods for 2024 can be found here: https://docs.google.com/spreadsheets/d/1nnYFcmhVieSHI5XAVhPiW1K6CoGYGxv2YPchwL0sqH4/edit?gid=0#gid=0

In [8]:
run_splits = []
header = []
with open("2024RunSplits.csv", 'r') as f:
    for i, line in enumerate(f):
        if i == 0: 
            header = line.split(',')
            continue
        
        split_dict = {}
        spline = line.split(',')
        for key, value in zip(header, spline):
            split_dict[key.strip()] = value.strip()   
        run_splits.append(split_dict)

with open("2024RunSplits.json", 'w') as f:
    json.dump(run_splits, f)